In [1]:
import time
import email
import pickle

import os; 
from os import listdir
from os.path import join, isdir

import pandas as pd
import swifter

import matplotlib.pyplot as plt

path = "/media/becode/3D_House/Enron_CSV/"

In [3]:
data = pd.read_csv(join(path, "reduced_mails_FINAL.csv"), index_col=0)
#data.to_csv(join(path, "reduced_mails_FINAL.csv"))

In [4]:
data

,Content,Date,From,To,Subject,X-Folder,Words,VecSum,Author,Lemmet,Corpus
254576,---------------------- Forwarded by Phillip K ...,2000-09-01 06:08:00,phillip.allen@enron.com,"mike.grigsby@enron.com, frank.ermis@enron.com",FYI,\Phillip_Allen_Dec2000\Notes Folders\Discussio...,"['Forwarded', 'Phillip', 'K', 'Allen', 'HOU', ...",145.607574,allen-p,"['Forwarded', 'Phillip', 'Allen', 'Enron', 'No...","[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1..."
254452,"Naomi,\n\nThe two analysts that I have had con...",1999-12-10 07:00:00,phillip.allen@enron.com,naomi.johnston@enron.com,NaN,\Phillip_Allen_Dec2000\Notes Folders\Discussio...,"['Naomi', 'The', 'two', 'analysts', 'I', 'cont...",-2.095150,allen-p,"['Naomi', 'analyst', 'contact', 'Lenhart', 'Vi...","[(0, 1), (7, 1), (10, 1), (11, 1), (12, 1), (1..."
254553,---------------------- Forwarded by Phillip K ...,2000-02-11 04:31:00,phillip.allen@enron.com,keith.holst@enron.com,RE: W basis quotes,\Phillip_Allen_Dec2000\Notes Folders\Discussio...,"['Forwarded', 'Phillip', 'K', 'Allen', 'HOU', ...",417.837555,allen-p,"['Forwarded', 'Phillip', 'Allen', 'George', 'R...","[(0, 2), (3, 1), (7, 2), (9, 2), (15, 2), (17,..."
254418,---------------------- Forwarded by Phillip K ...,2000-02-11 07:39:00,phillip.allen@enron.com,mike.grigsby@enron.com,Western Strategy Briefing,\Phillip_Allen_Dec2000\Notes Folders\Discussio...,"['Forwarded', 'Phillip', 'K', 'Allen', 'HOU', ...",116.993492,allen-p,"['Forwarded', 'Phillip', 'Allen', 'Heizenrader...","[(0, 2), (3, 1), (7, 2), (9, 1), (10, 1), (72,..."
254533,Dear Associate / Analyst Committee:\n\nThe fo...,2001-05-11 09:54:00,alyse.herasimchuk@enron.com,"phillip.allen@enron.com, robina.barker-bennett...","""Save the Date"" - Associate / Analyst Program",\Phillip_Allen_June2001\Notes Folders\Discussi...,"['Dear', 'Associate', 'Analyst', 'Committee', ...",50.773918,allen-p,"['Associate', 'Analyst', 'Committee', 'followi...","[(29, 1), (42, 1), (50, 1), (82, 3), (83, 3), ..."
...,...,...,...,...,...,...,...,...,...,...,...
48832,\nq2 traded away 32 watt ex no credit otc \nq...,2002-01-09 07:19:11,cramer@cadvision.com,john.zufferli@enron.com,trade tuesday jan 8,"\ExMerge - Zufferli, John\Inbox\Quotes","['q2', 'traded', 'away', '32', 'watt', 'ex', '...",31.143114,zufferli-j,"['trade', 'credit', 'trade', 'twice']","[(59, 1), (534, 2), (9060, 1)]"
48522,Enron Canada Power Corp acknowledges the chang...,2002-01-09 07:30:16,john.zufferli@enron.com,carol.moline@powerpool.ab.ca,Chevron Contract,"\ExMerge - Zufferli, John\Sent Items","['Enron', 'Canada', 'Power', 'Corp', 'acknowle...",116.660645,zufferli-j,"['Enron', 'Canada', 'Power', 'acknowledge', 'c...","[(2, 1), (15, 1), (22, 1), (87, 1), (141, 1), ..."
48781,This email is acknowledgement from the Power P...,2002-01-09 08:32:34,carol.moline@powerpool.ab.ca,slfr@chevrontexaco.com,Power Pool,"\ExMerge - Zufferli, John\Inbox","['This', 'email', 'acknowledgement', 'Power', ...",485.660370,zufferli-j,"['email', 'acknowledgement', 'Power', 'Alberta...","[(2, 4), (15, 5), (22, 5), (29, 1), (67, 1), (..."
48586,Nella here is a list of products that we would...,2002-01-09 09:46:43,john.zufferli@enron.com,nella.cappelletto@enron.com,Products,"\ExMerge - Zufferli, John\Sent Items","['Nella', 'list', 'products', 'would', 'launch...",28.633648,zufferli-j,"['Nella', 'product', 'would', 'launch', 'NETCO...","[(125, 1), (371, 1), (481, 3), (1441, 2), (177..."


In [43]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet

tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))

In [44]:
def get_lemma(word):
    lemma = wordnet.morphy(word)
    if lemma is None: return word
    else: return lemma

In [45]:
def preparation(email):
    tokens = tokenizer.tokenize(email)
    tokens = [t for t in tokens if len(t) > 4]
    tokens = [t for t in tokens if t not in stop_words]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [49]:
data["Lemmet"] = data["Content"].swifter.apply(preparation)

In [50]:
pd.Series(data["Lemmet"].apply(lambda x: len(x)))

254576    12
254452    31
254553    62
254418    17
254533    26
          ..
48832      4
48522     10
48781     85
48586     17
48559     43
Name: Lemmet, Length: 288337, dtype: int64

In [91]:
collection = []
for idx in data.index:
    collection.append(data.loc[idx, "Lemmet"])

print(sum(list(map(len, collection))), "words")

32251612 words


In [4]:
import gensim
from gensim import corpora

In [ ]:
dictionary = corpora.Dictionary(collection)

In [93]:
dictionary.save(join(path, "dictionary.gensim"))

In [5]:
dictionary = corpora.Dictionary.load(join(path, "dictionary.gensim"))

In [7]:
print(len(dictionary.keys()), "unique words in dictionary")

500372 unique words in dictionary


In [100]:
corpus = [dictionary.doc2bow(tokens) for tokens in collection]

In [109]:
pickle.dump(corpus, open(join(path, "corpus.pkl"), "wb"))

In [8]:
corpus = pickle.load(open(join(path, "corpus.pkl"), "rb"))

In [52]:
data["Corpus"] = corpus

In [41]:
forward_keys = [v[0] for v in dictionary.items() if "orward" in v[1]]

In [42]:
dictionary[forward_keys[0]]

'Forwarded'

In [11]:
import ast

In [12]:
ast.literal_eval(data["Words"].iloc[0])

['Forwarded',
 'Phillip',
 'K',
 'Allen',
 'HOU',
 'ECT',
 '09',
 '01',
 '2000',
 '01',
 '07',
 'PM',
 'Enron',
 'North',
 'America',
 'Corp',
 'From',
 'Matt',
 'Motley',
 '09',
 '01',
 '2000',
 '08',
 '53',
 'AM',
 'To',
 'Phillip',
 'K',
 'Allen',
 'HOU',
 'ECT',
 'ECT',
 'cc',
 'Subject',
 'FYI',
 'Ray',
 'Niles',
 'Price',
 'Caps',
 'pdf']

In [10]:
start = time.time()

data["Words"] = data["Words"].apply(ast.literal_eval)

print("Took", round(time.time() -start), "seconds.")

NameError: name 'ast' is not defined

In [ ]:
{word: data["Words"].iloc[0].count(word) for word in data["Words"].iloc[0]}

In [23]:
len(data)

288337

In [24]:
len(corpus)

288337

In [74]:
start = time.time()

model = gensim.models.ldamodel.LdaModel(corpus[:20], num_topics=3, id2word=dictionary, passes=20)

print("Took", round(time.time()-start,2), "seconds.")

Took 8.05 seconds.


In [177]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)

In [178]:
ldamodel.save(join(path, "model5.gensim"))

In [180]:
topics = ldamodel.print_topics(num_words=3)
for topic in topics:
    print(topic)

(0, '0.019*"would" + 0.011*"going" + 0.011*"think" + 0.008*"could" + 0.007*"still" + 0.007*"people" + 0.006*"money" + 0.005*"want" + 0.005*"around" + 0.005*"today"')
(1, '0.044*"IMAGE" + 0.011*"Click" + 0.009*"email" + 0.007*"click" + 0.006*"receive" + 0.006*"please" + 0.006*"Cannot" + 0.006*"online" + 0.005*"offer" + 0.005*"include"')
(2, '0.036*"database" + 0.028*"dbCaps97Data" + 0.019*"London" + 0.019*"Alias" + 0.018*"Scheduled" + 0.016*"Outages" + 0.012*"close" + 0.012*"perform" + 0.011*"operation" + 0.011*"system"')
(3, '0.020*"Enron" + 0.011*"company" + 0.010*"market" + 0.008*"power" + 0.007*"would" + 0.007*"energy" + 0.006*"price" + 0.006*"business" + 0.005*"state" + 0.005*"million"')
(4, '0.018*"Enron" + 0.012*"schedule" + 0.010*"please" + 0.009*"information" + 0.008*"enron" + 0.008*"employee" + 0.008*"Please" + 0.007*"access" + 0.007*"HourAhead" + 0.006*"Portland"')
(5, '0.046*"Subject" + 0.041*"Enron" + 0.030*"Original" + 0.030*"Message" + 0.018*"ENRON" + 0.016*"Thanks" + 0.0

In [ ]:
def topics_from_employee():
    df = reduced

In [127]:
data[data["Author"]=="allen-p"]["From"].value_counts()

phillip.allen@enron.com                            615
k..allen@enron.com                                 297
arsystem@mailman.enron.com                          37
no.address@enron.com                                30
webmaster@earnings.com                              23
                                                  ... 
editor@hersweeps.com                                 1
networkcommerce-tdcd20011221@ombramarketing.com      1
software@mail02.unitedmarketingstrategies.com        1
brad.jones@enron.com                                 1
mark.whitt@enron.com                                 1
Name: From, Length: 182, dtype: int64

In [128]:
all_mails = pd.read_csv(join(path, "all_mails.csv"), index_col=0)

In [131]:
all_mails[all_mails["X-Origin"]=="allen-p"]["From"].value_counts()

phillip.allen@enron.com           2125
k..allen@enron.com                 297
arsystem@mailman.enron.com          39
no.address@enron.com                30
webmaster@earnings.com              27
                                  ... 
c..aucoin@enron.com                  1
lisa@techxans.org                    1
ksumey@ftenergy.com                  1
prizemachine@feedback.iwon.com       1
infousa4492@telkom.net               1
Name: From, Length: 182, dtype: int64

## need to re-reduce the all_mails set to include every sent mail from employ

In [134]:
employee = pd.read_csv(join(path, "employees.csv"), index_col=0)

In [140]:
employee[employee["First"]=="Phillip"]

,Last,First,Mail
5,Allen,Phillip,"['phillip.allen@enron.com', 'k..allen@enron.com']"
114,Love,Phillip,"['m..love@enron.com', 'phillip.love@enron.com']"
143,Platter,Phillip,['phillip.platter@enron.com']


In [143]:
allen = all_mails[all_mails["From"].apply(lambda x: x in employee.loc[5, "Mail"])]

In [160]:
dct = {}
for origin in sorted(all_mails["X-Origin"].unique()):
    dct[origin] = all_mails[all_mails["X-Origin"]==origin]["From"].value_counts()

In [171]:
sum(list(map(lambda x: "allen" in x, list(dct["allen-p"].keys()))))

3